In [1]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## 建立共享函数

In [2]:
def weight(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1), name='W')

def bias(shape):
    return tf.Variable(tf.constant(0.1, shape=shape), name='b')

def conv2d(x,W):
    '''W是filter weight滤镜的权重,使用随机方式产生'''
    return tf.nn.conv2d(x,W, strides=[1,1,1,1], padding='SAME') #strides=[1,stride,stride,1],滤镜的步长

def max_pool_2x2(x):
    '''降采样'''
    return tf.nn.max_pool(x, ksize=[1,2,2,1],strides=[1,2,2,1], padding='SAME')

## 建立模型

In [3]:
with tf.name_scope('Input_Layer'):
    x = tf.placeholder('float',shape=[None, 784], name='x')
    x_image = tf.reshape(x, [-1,28,28,1])

In [4]:
#建立卷积层,以单个图像来产生多个图像,卷积运算后的效果类似于滤镜效果
with tf.name_scope('C1_Conv'):
    W1 = weight([5,5,1,16])
    b1 = bias([16])
    Conv1 = conv2d(x_image, W1)+b1
    C1_Conv = tf.nn.relu(Conv1)

In [5]:
#建立池化层1
with tf.name_scope('C1_Pool'):
    C1_Pool = max_pool_2x2(C1_Conv)

In [6]:
#建立卷积层2
with tf.name_scope('C2_Conv'):
    W2 = weight([5,5,16,36])
    b2 = bias([36])
    Conv2 = conv2d(C1_Pool, W2)+b2
    C2_Conv = tf.nn.relu(Conv2)

In [7]:
#建立池化层2
with tf.name_scope('C2_Pool'):
    C2_Pool = max_pool_2x2(C2_Conv)

In [8]:
#建立平坦层(因为卷积层是把数据变成二维的形式进行处理的,所以需要把二维数据展开成一维数据)
with tf.name_scope('D_Flat'):
    D_Flat = tf.reshape(C2_Pool, [-1, 1764]) #1764=36x7x7

In [9]:
#建立隐藏层
with tf.name_scope('D_Hidden_Layer'):
    W3 = weight([1764,128])
    b3 = bias([128])
    D_Hidden=tf.nn.relu(tf.matmul(D_Flat, W3)+b3)
    D_Hidden_Dropout = tf.nn.dropout(D_Hidden, keep_prob=0.8)

In [10]:
#建立输出层
with tf.name_scope('Output_Layer'):
    W4 = weight([128, 10])
    b4 = bias([10])
    y_predict = tf.nn.softmax(tf.matmul(D_Hidden_Dropout, W4)+b4)

## 定义训练方式

In [11]:
with tf.name_scope('optimizer'):
    y_label = tf.placeholder('float',shape=[None,10],name='y_label')
    loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_predict, labels=y_label))
    optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss_function)

## 定义评估模型准确率的方式

In [12]:
with tf.name_scope('evaluate_model'):
    correct_prediction = tf.equal(tf.argmax(y_predict, 1),
                                 tf.argmax(y_label, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))

## 进行训练
使用反向传播算法训练,训练数据共55000项,分为每一批次100项

In [13]:
trainEpochs = 30
batchSize = 100
totalBatchs = int(mnist.train.num_examples/batchSize)
epoch_list=[]
accuracy_list=[]
loss_list = []
from time import time
startTime = time()
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
for epoch in range(trainEpochs):
    for i in range(totalBatchs):
        batch_x, batch_y = mnist.train.next_batch(batchSize)
        sess.run(optimizer, feed_dict={x:batch_x, y_label:batch_y})
    loss, acc = sess.run([loss_function,accuracy],feed_dict = {x:mnist.validation.images, y_label:mnist.validation.labels})
    epoch_list.append(epoch)
    accuracy_list.append(acc)
    loss_list.append(loss)
    print('Train Epoch:', "%02d"%(epoch+1),'Loss=',"{:.9f}".format(loss), " Accuracy=", acc)

('Train Epoch:', '01', 'Loss=', '1.667765975', ' Accuracy=', 0.81040001)


In [ ]:
duration = time()-startTime
print('Train Finished takes:', duration)

In [ ]:
print('Accuracy:', sess.run(accuracy, feed_dict={x:mnist.test.images, y_label:mnist.test.labels}))

## 进行预测

In [ ]:
prediction_result = sess.run(tf.argmax(y_predict, 1),
                            feed_dict={x:mnist_test.images, y_label:mnist.test.labels})

In [ ]:
prediction_result[:10]

In [ ]:
def plot_images_labels_prediction(images, labels, prediction, idx, num=10):
    fig = plt.gcf()
    fig.set_size_inches(12,14)
    if num>25: num=25
    
    for i in range(0, num):
        ax = plt.subplot(5,5, 1+i)
        ax.imshow(np.reshape(images[idx], (28,28)), cmap='binary')
        title = 'label='+str(np.argmax(labels[idx]))
        
        if len(prediction)>0:
            title +=", predict="+str(prediction[idx])
            
        ax.set_title(title, fontsize=10)
        ax.set_xticks([]);ax.set_yticks([])
        idx+=1
    plt.show()

## TensorBoard

In [ ]:
merged=tf.summary.merge_all()
train_writer = tf.summary.FileWriter('log/CNN', sess.graph)

In [ ]:
!tensorboard --logdir=./log/CNN